In [120]:
#import functions
import torch
import torch.nn as nn
import numpy as np
import matplotlib.pyplot as plt
import math
import pandas as pd
import torch.nn.functional as F
import random
import torch.optim as optim
from scipy.io import loadmat
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch.utils.data import DataLoader, TensorDataset
import scipy.io

In [121]:
from torch.optim.lr_scheduler import ReduceLROnPlateau

# Import data

below I have data that represents time invariant or time variant data. to test 1 set of data simply uncomment the other set of data

In [ ]:
'''
Time invariant data is below
'''

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\tlds.csv"
#tlds = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\tlgs.csv"
#tlgs = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\taus.csv"
#taus = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\kps.csv"
#kps = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\pilotoutput.csv"
#p = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\pitcherror.csv"
#pitch_error = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\pitchcmd.csv"
#pitch_cmd = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\pitchoutput.csv"
#pitch_out = torch.tensor((pd.read_csv(file_path)).values)

#file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\invariant_cmd3_cessna\xpilot.csv"
#xp = torch.tensor((pd.read_csv(file_path)).values)


'''
Time variant data below
'''
file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\tlds.csv"
tlds = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\tlgs.csv"
tlgs = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\taus.csv"
taus = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\kps.csv"
kps = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\pilotoutput.csv"
p = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\pitcherror.csv"
pitch_error = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\pitchcmd.csv"
pitch_cmd = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\pitchoutput.csv"
pitch_out = torch.tensor((pd.read_csv(file_path)).values)

file_path = r"C:\Users\steph\OneDrive\Desktop\Stephen\ADCL\thesis stuff\ALL THESIS DATA\large data PINN model data\Data for time domain pinn\theta_cmd_3_pilot_simulator\variant_cmd3_cessna\xpilot.csv"
xp = torch.tensor((pd.read_csv(file_path)).values)

Ts = 0.1
Tfinal = 100
length = Tfinal/Ts
time = torch.arange(0, Tfinal + Ts, Ts)[:-1].view(-1,1)

# Data analysis/cleaning

In [123]:
# Check if each column of tensor1 is larger than the corresponding column of tensor2
is_column_larger = tlgs > tlds

# Check if all columns of tensor1 are larger than corresponding columns of tensor2
are_all_columns_larger = torch.all(is_column_larger, dim=0)

print("Are all columns of tlgs larger than corresponding columns of tlds:", torch.all(are_all_columns_larger).item())

# Threshold value
threshold = 1.7

# Find the maximum value of each column
max_values, _ = torch.max(pitch_out, dim=0)

# Obtain the column numbers where the maximum value exceeds the threshold
columns_above_threshold = torch.nonzero(max_values > threshold).squeeze()

print("Column numbers where max value exceeds the threshold:", columns_above_threshold.tolist())

# Create a mask to select columns to keep
mask = torch.ones(pitch_out.size(1), dtype=torch.bool)
mask[columns_above_threshold] = False

# Delete columns based on the mask
pitch_out_filtered = pitch_out[:, mask]


tlds_filtered = tlds[:,mask]
tlgs_filtered = tlgs[:,mask]
taus_filtered = taus[:,mask]
kps_filtered=kps[:,mask]
p_filtered = p[:,mask]
pitch_cmd_filtered = pitch_cmd[:,mask]
pitch_error_filtered = pitch_error[:,mask]
xp_filtered = xp[:,mask]

pitch_out_filtered.shape, tlds_filtered.shape, tlgs_filtered.shape, kps_filtered.shape,p_filtered.shape, pitch_cmd_filtered.shape,pitch_error_filtered.shape,xp_filtered.shape



Are all columns of tlgs larger than corresponding columns of tlds: False
Column numbers where max value exceeds the threshold: [98, 132, 137, 145, 227, 235, 272, 288, 371, 384, 604]


(torch.Size([1000, 739]),
 torch.Size([1000, 739]),
 torch.Size([1000, 739]),
 torch.Size([1000, 739]),
 torch.Size([1000, 739]),
 torch.Size([1000, 739]),
 torch.Size([1000, 739]),
 torch.Size([1000, 739]))

## convert data to tensors

In [124]:
tlds_tensor = torch.tensor(tlds_filtered, dtype=torch.float32)
tlgs_tensor = torch.tensor(tlgs_filtered, dtype=torch.float32)
taus_tensor = torch.tensor(taus_filtered, dtype=torch.float32)
kps_tensor = torch.tensor(kps_filtered, dtype=torch.float32)
p_tensor = torch.tensor(p_filtered, dtype=torch.float32)
pitch_error_tensor = torch.tensor(pitch_error_filtered, dtype=torch.float32)
pitch_cmd_tensor = torch.tensor(pitch_cmd_filtered, dtype=torch.float32)
pitch_out_tensor = torch.tensor(pitch_out_filtered, dtype=torch.float32)
xp_tensor = torch.tensor(xp_filtered, dtype=torch.float32)
time_tensor = torch.tensor(time, dtype=torch.float32)

C:\Users\steph\AppData\Local\Temp\ipykernel_31832\2570946064.py:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tlds_tensor = torch.tensor(tlds_filtered, dtype=torch.float32)
C:\Users\steph\AppData\Local\Temp\ipykernel_31832\2570946064.py:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  tlgs_tensor = torch.tensor(tlgs_filtered, dtype=torch.float32)
C:\Users\steph\AppData\Local\Temp\ipykernel_31832\2570946064.py:3: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  taus_tensor = torch.tensor(taus_filtered, dtype=torch.float32)
C:\Users\steph\A

## normalize the data

In [125]:
pitch_cmd_tensor_norm = (pitch_cmd_tensor - pitch_cmd_tensor.min(dim=0).values) / (pitch_cmd_tensor.max(dim=0).values - pitch_cmd_tensor.min(dim=0).values)
pitch_out_tensor_norm = (pitch_out_tensor - pitch_out_tensor.min(dim=0).values) / (pitch_out_tensor.max(dim=0).values - pitch_out_tensor.min(dim=0).values)
pitch_error_tensor_norm = (pitch_error_tensor - pitch_error_tensor.min(dim=0).values) / (pitch_error_tensor.max(dim=0).values - pitch_error_tensor.min(dim=0).values)
pilot_output_tensor_norm = (p_tensor - p_tensor.min(dim=0).values) / (p_tensor.max(dim=0).values - p_tensor.min(dim=0).values)
time_norm = (time - time.min(dim=0).values) / (time.max(dim=0).values - time.min(dim=0).values)

In [126]:
length = 1000

## create X

In [127]:
tensor_list = []
for i in range(tlds_tensor.shape[1]):
  tensor_2d = (torch.cat((pitch_cmd_tensor_norm[:length,i].view(-1,1), pitch_out_tensor_norm[:length,i].view(-1,1),pitch_error_tensor_norm[:length,i].view(-1,1),pilot_output_tensor_norm[:length,i].view(-1,1), time_norm[:length]),dim=1))
  tensor_list.append(tensor_2d)
X = torch.stack(tensor_list)
X.shape #this is in the shape of (# of runs, data points, # of features)

torch.Size([739, 1000, 5])

## create y

In [128]:
tlds_tensor_2 =  tlds_tensor.T
tlgs_tensor_2 = tlgs_tensor.T
taus_tensor_2 = taus_tensor.T
kps_tensor_2 = kps_tensor.T
tlds_tensor_2.shape

tensor_list = []
for j in range(tlds_tensor.shape[1]):
  row_tensor = torch.concat((xp_tensor.T[j,:length].view(1,-1),tlds_tensor_2[j,:length].view(1,-1),tlgs_tensor_2[j,:length].view(1,-1),taus_tensor_2[j,:length].view(1,-1), kps_tensor_2[j,:length].view(1,-1)),dim=1)
  tensor_list.append(row_tensor)

y = torch.stack(tensor_list)
y=torch.squeeze(y)
y.shape

torch.Size([739, 5000])

In [129]:
pitch_error_tensor.T.shape, p_tensor.T.shape

(torch.Size([739, 1000]), torch.Size([739, 1000]))

In [130]:
pitch_error_tensor_reshape = pitch_error_tensor.T
p_tensor_reshape = p_tensor.T
pitch_error_tensor_reshape.shape, p_tensor_reshape.shape

(torch.Size([739, 1000]), torch.Size([739, 1000]))

## train/test split

In [131]:
import sklearn
from sklearn.model_selection import train_test_split

Xtrain, Xtest, pitcherrortrain, pitcherrortest, ptrain, ptest, ytrain, ytest = train_test_split(
    X, pitch_error_tensor_reshape, p_tensor_reshape, y,
    test_size=0.2,        # Proportion of the dataset to include in the test split
    random_state=123       # Seed for the random number generator
)

Xtrain.shape, Xtest.shape, pitcherrortrain.shape, pitcherrortest.shape, ptrain.shape, ptest.shape, ytrain.shape, ytest.shape

(torch.Size([591, 1000, 5]),
 torch.Size([148, 1000, 5]),
 torch.Size([591, 1000]),
 torch.Size([148, 1000]),
 torch.Size([591, 1000]),
 torch.Size([148, 1000]),
 torch.Size([591, 5000]),
 torch.Size([148, 5000]))

# Create models

below is the RNN, LSTM, CNN models. comment out whichever u dont need

## RNN model

In [ ]:
# RNN model
class BasicRNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BasicRNNModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, num_layers, batch_first=True, nonlinearity="tanh")

        # Define fully connected layers for the output
        self.fc1 = nn.Linear(hidden_size, hidden_size)  # Hidden size reduced by half
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer

        # ReLU activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Initialize hidden state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # hidden state

        # Forward pass through RNN layer
        out, _ = self.rnn(x, h0)

        # Take the output from the last time step
        out = out[:, -1, :]  # Shape: (batch_size, hidden_size)

        # Apply ReLU activation
        out = self.relu(out)

        # Pass through the first fully connected layer
        out = self.fc1(out)
        out = self.relu(out)

        # Pass through the second fully connected layer (output layer)
        out = self.fc2(out)

        return out

# Example Usage:
# Define parameters
input_size = 5       # Number of input features
hidden_size = 64     # Number of hidden units
output_size = 5000   # Output dimension
num_layers = 1       # Number of RNN layers

# Create an instance of the model
model = BasicRNNModel(input_size, hidden_size, output_size, num_layers)

## LSTM model

In [ ]:
#LSTM model

class BasicLSTMModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BasicLSTMModel, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers

        # Define LSTM layer
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first=True)

        # Define fully connected layers for the output
        self.fc1 = nn.Linear(hidden_size, hidden_size)  # Hidden size reduced by half
        self.fc2 = nn.Linear(hidden_size, output_size)  # Output layer

        # ReLU activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Initialize hidden state and cell state
        h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # hidden state
        c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(x.device)  # cell state

        # Forward pass through LSTM layer
        out, _ = self.lstm(x, (h0, c0))

        # Take the output from the last time step
        out = out[:, -1, :]  # Shape: (batch_size, hidden_size)

        # Apply ReLU activation
        out = self.relu(out)

        # Pass through the first fully connected layer
        out = self.fc1(out)
        out = self.relu(out)

        # Pass through the second fully connected layer (output layer)
        out = self.fc2(out)

        return out

# Example Usage:
# Define parameters
input_size = Xtrain.shape[2]   # Number of input features
hidden_size = 64  # Number of hidden units
output_size = ytrain.shape[1]   # Output dimension (for regression, binary classification, etc.)
num_layers = 1    # Number of LSTM layers

# Create an instance of the model
model = BasicLSTMModel(input_size, hidden_size, output_size, num_layers)




## CNN model

In [ ]:
# CNN model

class BasicCNNModel(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers=1):
        super(BasicCNNModel, self).__init__()
        
        # Convolutional Layer
        self.conv1 = nn.Conv1d(in_channels=input_size, out_channels=hidden_size, kernel_size=10, padding=1)
        
        # Define a fully connected hidden layer
        self.fc_hidden = nn.Linear(hidden_size, hidden_size)
        
        # ReLU activation
        self.relu = nn.ReLU()
        
        # Output layer
        self.fc_output = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        # (Batch, Seq_len, Features) -> (Batch, Features, Seq_len) for Conv1d
        x = x.permute(0, 2, 1)
        
        # Apply Convolutional layer
        out = self.conv1(x)
        
        # Apply ReLU activation
        out = self.relu(out)
        
        # Apply Global Max Pooling (collapse sequence dimension)
        out, _ = torch.max(out, dim=2)
        
        # Pass through the fully connected hidden layer
        out = self.fc_hidden(out)
        out = self.relu(out)

        # Pass through the output layer
        out = self.fc_output(out)
        
        # Apply sigmoid scaling to specific indices
        #out[:, 750:1000] = torch.sigmoid(out[:, 750:1000]) * 0.9 + 0.1  # Scale between 0.1 and 1

        return out

# Example Usage:
# Define parameters
input_size = Xtrain.shape[2]   # Number of input features
hidden_size = 128  # Number of hidden units (out_channels for CNN)
output_size = ytrain.shape[1]   # Output dimension (for regression, binary classification, etc.)
num_layers = 1    # (Not needed for CNN, but kept to mirror LSTM)

# Create an instance of the CNN model
model = BasicCNNModel(input_size, hidden_size, output_size, num_layers)

# create data loader

In [133]:
from torch.utils.data import Dataset, DataLoader
torch.manual_seed(123)
# Create a custom dataset
class CustomDataset(Dataset):
    def __init__(self, X_train, y_train, additional_data1, additional_data2):
        self.X_train = X_train
        self.y_train = y_train
        self.additional_data1 = additional_data1
        self.additional_data2 = additional_data2

    def __len__(self):
        return len(self.X_train)

    def __getitem__(self, idx):
        x = self.X_train[idx]
        y = self.y_train[idx]
        additional1 = self.additional_data1[idx]
        additional2 = self.additional_data2[idx]
        return x, y, additional1, additional2

# Define batch size
batch_size = 32

# Create the custom dataset
custom_dataset = CustomDataset(X_train=Xtrain, y_train=ytrain, additional_data1=ptrain, additional_data2=pitcherrortrain)

# Create DataLoader with shuffle=False
train_loader = DataLoader(custom_dataset, batch_size=batch_size, shuffle=False)


In [134]:
torch.manual_seed(123)

# define the optimiser
optimiser = torch.optim.Adam(model.parameters(), lr=1e-3)

# define the number of epochs
epochs = 50

# collect the loss history
total_loss_history=[]
pilot_output_loss_history=[]
xpilot_calcualted_loss_history=[]
params_loss_history=[]


# define the loss criteria
criterion= nn.MSELoss()

In [135]:
import torch
import matplotlib.pyplot as plt
import numpy as np

def transport_delay(data, tau, time_step, initial_value):
    """
    Applies a transport delay to a 1D tensor of time series data.

    Parameters:
    - data: 1D PyTorch tensor containing the time series data.
    - tau: The delay in seconds.
    - time_step: The time step between consecutive samples in seconds.
    - initial_value: The value to fill in the delayed portion of the tensor.

    Returns:
    - delayed_data: The tensor with the applied delay.
    """
    if not isinstance(data, torch.Tensor):
        raise ValueError("data should be a PyTorch tensor")

    if data.dim() != 1:
        raise ValueError("data should be a 1D tensor")

    # Calculate the number of samples to delay
    #delay_steps = int(round(tau / time_step))
    delay_steps = (int((tau / time_step)))

    # Ensure delay_steps is non-negative
    delay_steps = max(delay_steps, 0)

    # Create a tensor with the same shape as data, filled with initial_value
    delayed_data = torch.full_like(data, initial_value)

    # Apply the delay by shifting the data tensor
    if delay_steps < len(data):
        delayed_data[delay_steps:] = data[:-delay_steps]
    else:
        # If delay_steps is larger than the data length, return all initial values
        delayed_data = torch.full_like(data, initial_value)

    return delayed_data

# training loop

for this paper we tried 2 different training loops, one without physics informed (top code), and 1 with physics informed(bottom code). uncomment the 1 u dont need

## no physics informed

In [ ]:

#from torch.optim.lr_scheduler import ReduceLROnPlateau
#scheduler = ReduceLROnPlateau(optimiser, 'min', patience=len(train_loader)*3, factor=0.1)

torch.manual_seed(123)

for epoch in range(epochs):

    model.train()

    tensor_list = []
    xpilot_list=[]

    for batch_X, batch_y, batch_additional1, batch_additional2 in train_loader:

      # Zero the gradients
      optimiser.zero_grad()

      # Forward pass
      xp_est, params_est = model(batch_X)[:,:1000], model(batch_X)[:,1000:]
      
      # Compute Pilot Parameters Loss
      params_loss = criterion(params_est, batch_y[:,1000:])


      # Compute X_pilot loss
      xpilot_loss = criterion(xp_est[:,:1000], batch_y[:,:1000])

      # Compute Total Loss
      loss = xpilot_loss + params_loss

      total_loss_history.append(loss.item())

      # Backward pass
      loss.backward()

      # Update step
      optimiser.step()
      
    # Print results
    print(f"Epoch {epoch+1}/{epochs}, Total Loss: {loss.item():.4f}")

#if epoch >20:
#scheduler.step(loss)

Epoch 1/50, Total Loss: 0.2490
Epoch 2/50, Total Loss: 0.0637
Epoch 3/50, Total Loss: 0.0596
Epoch 4/50, Total Loss: 0.0572
Epoch 5/50, Total Loss: 0.0567
Epoch 6/50, Total Loss: 0.0561
Epoch 7/50, Total Loss: 0.0559
Epoch 8/50, Total Loss: 0.0555
Epoch 9/50, Total Loss: 0.0552
Epoch 10/50, Total Loss: 0.0547
Epoch 11/50, Total Loss: 0.0549
Epoch 12/50, Total Loss: 0.0539
Epoch 13/50, Total Loss: 0.0524
Epoch 14/50, Total Loss: 0.0507
Epoch 15/50, Total Loss: 0.0484
Epoch 16/50, Total Loss: 0.0454
Epoch 17/50, Total Loss: 0.0427
Epoch 18/50, Total Loss: 0.0410
Epoch 19/50, Total Loss: 0.0392
Epoch 20/50, Total Loss: 0.0374
Epoch 21/50, Total Loss: 0.0358
Epoch 22/50, Total Loss: 0.0342
Epoch 23/50, Total Loss: 0.0326
Epoch 24/50, Total Loss: 0.0311
Epoch 25/50, Total Loss: 0.0300
Epoch 26/50, Total Loss: 0.0294
Epoch 27/50, Total Loss: 0.0289
Epoch 28/50, Total Loss: 0.0283
Epoch 29/50, Total Loss: 0.0278
Epoch 30/50, Total Loss: 0.0272
Epoch 31/50, Total Loss: 0.0267
Epoch 32/50, Tota

## with physics informed

In [ ]:
from torch.optim.lr_scheduler import ReduceLROnPlateau
#scheduler = ReduceLROnPlateau(optimiser, 'min', patience=len(train_loader)*10, factor=0.1, verbose=True)


torch.manual_seed(123)

switch_epoch = 10
for epoch in range(epochs):

    model.train()

    tensor_list = []
    xpilot_list=[]

    for batch_X, batch_y, batch_additional1, batch_additional2 in train_loader:

      # Zero the gradients
      optimiser.zero_grad()

      # Forward pass
      xp_est, params_est = model(batch_X)[:,:1000], model(batch_X)[:,-4000:]

      # Compute X_pilot loss
      xpilot_loss = criterion(xp_est[:,:300], batch_y[:,:300])

      # Compute parameters loss
      #lambda_reg = 0.1
      #tld_loss = criterion(params_est[:,0:1000], batch_y[:,1000:2000]) #+ lambda_reg*torch.sum((params_est[:,0:1000]- batch_y[:,1000:2000])**2)
      #tlg_loss = criterion(params_est[:,1000:2000],batch_y[:,2000:3000]) #+ lambda_reg*torch.sum((params_est[:,1000:2000]- batch_y[:,2000:3000])**2)
      #tau_loss = criterion(params_est[:,2000:3000],batch_y[:,3000:4000])
      #kp_loss = criterion(params_est[:,3000:],batch_y[:,4000:])
      #params_loss = tld_loss + tlg_loss + tau_loss + kp_loss
      params_loss = criterion(params_est, batch_y[:,-4000:])
      params_loss_history.append(params_loss.item())

      if epoch >= switch_epoch:
        # Compute physics losses
        for x in range(batch_X.shape[0]):
          u = xp_est[x,:]
          tld_t,tlg_t,tau_t,kp_t = params_est[x,:1000],params_est[x,1000:2000],params_est[x,2000:3000], params_est[x,3000:4000]
          if tau_t.mean() < 0.1:
            tau_t_use = 0.1
          else:
            tau_t_use = tau_t.mean()
          pitcherror_t_tau = transport_delay(batch_additional2[x,:], tau_t_use, 0.1, batch_additional2[x,:][0])

          # Calculate estimated pilot output
          pilot_output_estimated = ((-kp_t/(tlg_t*tlg_t))*(tlg_t-tld_t)*u - (kp_t*tld_t/tlg_t)*pitcherror_t_tau).view(1,-1)
          tensor_list.append(pilot_output_estimated)

          # Calculate estimated x pilot
          xpilot_calcualted = ((pitcherror_t_tau)/tlg_t) - ((pitcherror_t_tau/tlg_t)*(torch.exp(-time.squeeze()/tlg_t))).reshape(1,-1)
          xpilot_list.append(xpilot_calcualted)


        # Compute pilot output loss
        pilot_output_concat = torch.cat(tensor_list, dim=0)
        pilot_output_loss = criterion(pilot_output_concat,batch_additional1)
        pilot_output_loss_history.append(pilot_output_loss.item())
        tensor_list.clear()

        # Compute calculated x pilot loss
        xpilot_calcualted_concat = torch.cat(xpilot_list,dim=0)
        xpilot_calculated_loss = criterion(xpilot_calcualted_concat,batch_y[:,:1000])
        xpilot_calcualted_loss_history.append(xpilot_calculated_loss)
        xpilot_list.clear()


      if epoch < switch_epoch:
        loss = xpilot_loss + params_loss
      else:
        loss = xpilot_loss + params_loss + pilot_output_loss + xpilot_calculated_loss
      total_loss_history.append(loss.item())


      # Backward pass
      loss.backward()


      # Update step
      optimiser.step()
    # Print results
    print(f"Epoch {epoch+1}/{epochs}, Total Loss: {loss.item():.4f}")

    # Step the scheduler
    #scheduler.step(loss)


# testing and prediction metrics

In [137]:
window_size = 1000
model.eval()

with torch.no_grad():

  # grab estimated params
  dense_model_preds = model(Xtest)
  estimated_params = dense_model_preds[:,window_size:]
  tld_est,tlg_est,tau_est,kp_est = estimated_params[:,:window_size],estimated_params[:,window_size:2*window_size],estimated_params[:,2*window_size:3*window_size],estimated_params[:,3*window_size:4*window_size]


  # grab true params
  true_params = ytest[:,window_size:]
  tld_true,tlg_true,tau_true,kp_true = true_params[:,:window_size],true_params[:,window_size:2*window_size], true_params[:,2*window_size:3*window_size],true_params[:,3*window_size:4*window_size]

  from sklearn.metrics import mean_absolute_error, mean_squared_error
  tld_mse = mean_squared_error(tld_true,tld_est)
  tlg_mse = mean_squared_error(tlg_true,tlg_est)
  tau_mse = mean_squared_error(tau_true,tau_est)
  kp_mse = mean_squared_error(kp_true,kp_est)

tld_mse, tlg_mse, tau_mse, kp_mse

(0.024960136, 0.07758582, 0.0041543623, 0.006745785)

In [138]:
xp_true = ytest[:,:window_size]
estimated_xp = dense_model_preds[:,:window_size]
mean_squared_error(xp_true,estimated_xp)

0.0004916807

In [139]:
# Initialize an empty list to store the tensors
pilot_output_predicted_list = []

for k in range(len(Xtest)):
  tld_est_int = tld_est[k,:]
  tlg_est_int = tlg_est[k,:]
  tau_est_int = tau_est[k,:]
  tau_est_avg = tau_est_int.mean()
  kp_est_int = kp_est[k,:]

  pitcherror_t_tau = transport_delay(pitcherrortest[k,:], tau_est_avg, 0.1, pitcherrortest[k,:][0])
  pilot_output_predicted = (-kp_est_int/(tlg_est_int*tlg_est_int)) * (tlg_est_int-tld_est_int) * estimated_xp[k,:] - (kp_est_int*tld_est_int/tlg_est_int)*pitcherror_t_tau

  # Append to the list
  pilot_output_predicted_list.append(pilot_output_predicted)


# Convert the list to a 2D tensor (NumPy array)
pilot_output_predicted_tensor = np.stack(pilot_output_predicted_list)

mean_squared_error(ptest,pilot_output_predicted_tensor)

1.3651288e-05

# save the model

In [ ]:
#torch.save(model.state_dict(), 'scitechpaper1_cnn_with_physics.pth') # insert appropiate model name